In [ ]:
!ls /datasets/cms

Medicare_Physician_Other_Practitioners_by_Provider_and_Service_2019.csv
Medicare_Physician_Other_Practitioners_by_Provider_and_Service_2019.zip
MUP_PHY_R19_P04_V10_D13_Geo.csv
MUP_PHY_R19_P04_V10_D14_Geo.csv
MUP_PHY_R19_P04_V10_D15_Geo.csv
MUP_PHY_R19_P04_V10_D16_Geo.csv
MUP_PHY_R19_P04_V10_D17_Geo.csv
MUP_PHY_R20_P04_V10_D18_Geo.csv
MUP_PHY_R21_P04_V10_D19_Geo.csv
prov_svc_2019_mi.parquet
SC-EST2020-ALLDATA5.csv
SC-EST2020-ALLDATA6.csv


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet('/datasets/cms/prov_svc_2019_mi.parquet')
# dtype={'HCPCS_Cd': 'string', 'Rndrng_Prvdr_Zip5': 'string','Rndrng_Prvdr_RUCA': 'float64', 'Rndrng_Prvdr_State_FIPS': 'float64', 'Tot_Srvcs': 'float64'})
# df.dtypes

In [ ]:
# missing value for Rndrng_Prvdr_Crdntls and Rndrng_Prvdr_Gndr
df = df.fillna('')

In [ ]:
def crdntls_map(crdntls):
    result = crdntls
    if (type(crdntls) == type('str')):        
        result = crdntls.replace ('.','').replace('>','') \
                        .replace('DOCTOR OF AUDIOLOGY','AUD').replace('DR  AUDIOLOGY','AUD').replace('AUDIOLOGIST','AUD') \
                        .replace('DC CHIROPRACTOR','DC').replace('DOCTOR OF CHIROPRACT','DC').replace('CHIROPRACTOR DC','DC')  \
                        .replace('DC DR OF CHIROPRACTI','DC').replace('DC DOCTOR OF CHIROPR','DC').replace('CHIROPRACTIC','DC')  \
                        .replace('D O','DO').replace('D.O.','DO').replace('D.O','DO') \
                        .replace('D P M','DPM') \
                        .replace ('M D','MD').replace ('M.D.','MD').replace ('M. D.','MD').replace('M.D','MD').replace('M>D>','MD') \
                        .replace('M D','MD').replace('M,D,','MD') \
                        .replace('N P','NP') \
                        .replace('NURSE PRACTITIONER','NP').replace('NURSE PRACITIONER','NP').replace('NP NURSE PRACTITIONE','NP') \
                        .replace('NURSE PRACTIONER','NP').replace('NP- NURSE PRACTITION', 'NP').replace('NURSE PRACTTIONER','NP') \
                        .replace('NP(WOMEN\'S HEALTH)','NP').replace('FAMILY NURSE PRACTIT','NP').replace('FAMILY NURSE PRACTIO', 'NP') \
                        .replace('NATALIE BROWN','').replace('ISLAM GOMAA','') \
                        .replace('FAMILY PRACTICE','').replace('INTERNAL MEDICINE','') \
                        .replace('OD DR OF OPTOMETRY','OD').replace('LINDSEY WYNKOOP, OD','OD')  \
                        .replace('PA-C','PAC').replace('(PA-C)','PAC').replace('(PAC)','PAC').replace('PA -C','PAC') \
                        .replace('PH D','PHD').replace('PH.D.','PHD').replace('DOCTORATE IN PHYSICA','PHD')  \
                        .replace('PHYSICIAN ASSISTANT','PA').replace('PHYSICIANS ASSISTANT','PA').replace('WILLIAM BLAKESLEE PA','PA') \
                        .replace('P A','PA').replace('PHYSICAL THERAPIST', 'PT').replace('P T','PT') \
                        .replace('REGISTERED DIETITIAN','RD')  \
                        .replace('SOCIAL WORKER','SW')
    else:
        result = ''
    
    return result

def isMD(crdntls):
    result = 0
    if (('MD' in crdntls) or ('DO' in crdntls) or ('MBBS' in crdntls)):
        result = 1
    return result

df.Rndrng_Prvdr_Crdntls = df.Rndrng_Prvdr_Crdntls.apply(lambda x: crdntls_map(x))
df['md'] = df.Rndrng_Prvdr_Crdntls.apply(lambda x: isMD(x))

In [ ]:
# len(df.Rndrng_Prvdr_Crdntls.unique()), df.Rndrng_Prvdr_Crdntls.unique()
# df.Rndrng_Prvdr_Gndr.unique()
# df.Rndrng_Prvdr_Ent_Cd.unique()
# df.Rndrng_Prvdr_Type.unique()
# df.Rndrng_Prvdr_Mdcr_Prtcptg_Ind.unique()
# df.HCPCS_Drug_Ind.unique()
# df.Place_Of_Srvc.unique()

In [ ]:
# feature engineering: turn category data to numberic

df['gender'] = np.where((df.Rndrng_Prvdr_Gndr == 'M'), 1, 0) 
df['entity'] = np.where(df.Rndrng_Prvdr_Ent_Cd == 'O', 1, 0) # organization or individual
df['Rndrng_Prvdr_Mdcr_Prtcptg_Ind'] = np.where(df.Rndrng_Prvdr_Mdcr_Prtcptg_Ind == 'Y', 1, 0)
df['HCPCS_Drug_Ind'] = np.where(df.HCPCS_Drug_Ind == 'Y', 1, 0)
df['facility'] = np.where(df.Place_Of_Srvc == 'F', 1, 0)

In [ ]:
# len(df.md.unique()), len(df.gender.unique()), len(df.entity.unique()), len(df.facility.unique())

In [ ]:
# df.Rndrng_Prvdr_RUCA.unique()
# df.groupby('Rndrng_Prvdr_RUCA').count()

df['Rndrng_Prvdr_RUCA'] = pd.to_numeric(df['Rndrng_Prvdr_RUCA'])
df['metro'] = np.where((df.Rndrng_Prvdr_RUCA <=6), 1, 0)

# df.groupby('metro').count()

In [ ]:
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('A'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('C'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('G'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('J'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('P'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('Q'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('R'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('V'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.endswith('T'), df.HCPCS_Cd.str[:-2], df.HCPCS_Cd)
# df['HCPCS_Cd'] = pd.to_numeric(df['HCPCS_Cd'])

In [ ]:
df['Tot_Benes'] = pd.to_numeric(df['Tot_Benes'])
df['Tot_Srvcs'] = pd.to_numeric(df['Tot_Srvcs'])
df['Tot_Bene_Day_Srvcs'] = pd.to_numeric(df['Tot_Bene_Day_Srvcs'])
df['Avg_Sbmtd_Chrg'] = pd.to_numeric(df['Avg_Sbmtd_Chrg'])
df['Avg_Mdcr_Alowd_Amt'] = pd.to_numeric(df['Avg_Mdcr_Alowd_Amt'])
df['Avg_Mdcr_Pymt_Amt'] = pd.to_numeric(df['Avg_Mdcr_Pymt_Amt'])
df['Avg_Mdcr_Stdzd_Amt'] = pd.to_numeric(df['Avg_Mdcr_Stdzd_Amt'])

In [ ]:
df = df.drop(columns=['Rndrng_NPI', 'Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name', 'Rndrng_Prvdr_MI', \
                'Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2', 'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn', \
                'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA_Desc', 'Rndrng_Prvdr_Cntry', \
                'Rndrng_Prvdr_Type', 'HCPCS_Cd', 'HCPCS_Desc'])

# these 2 are highly related to Avg_Mdcr_Alowd_Amt over 99%
df = df.drop(columns=['Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt'])

# following columns converted to new binary column
df = df.drop(columns=['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr', 'Rndrng_Prvdr_Ent_Cd', 'Rndrng_Prvdr_RUCA', 'Place_Of_Srvc'])


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337351 entries, 280 to 10139999
Data columns (total 12 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Rndrng_Prvdr_Mdcr_Prtcptg_Ind  337351 non-null  int64  
 1   HCPCS_Drug_Ind                 337351 non-null  int64  
 2   Tot_Benes                      337351 non-null  int64  
 3   Tot_Srvcs                      337351 non-null  float64
 4   Tot_Bene_Day_Srvcs             337351 non-null  int64  
 5   Avg_Sbmtd_Chrg                 337351 non-null  float64
 6   Avg_Mdcr_Alowd_Amt             337351 non-null  float64
 7   md                             337351 non-null  int64  
 8   gender                         337351 non-null  int64  
 9   entity                         337351 non-null  int64  
 10  facility                       337351 non-null  int64  
 11  metro                          337351 non-null  int64  
dtypes: float64(3), int64(9)
me

In [ ]:
df.describe()

,Rndrng_Prvdr_Mdcr_Prtcptg_Ind,HCPCS_Drug_Ind,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,md,gender,entity,facility,metro
count,337351.000000,337351.000000,337351.000000,3.373510e+05,337351.000000,337351.000000,337351.000000,337351.000000,337351.000000,337351.000000,337351.000000,337351.000000
mean,0.999864,0.056348,69.886279,1.799152e+02,112.405637,320.321263,99.076016,0.739793,0.665052,0.032183,0.441887,0.958462
std,0.011676,0.230593,267.017644,3.143331e+03,488.840097,923.623521,284.014269,0.438748,0.471973,0.176486,0.496612,0.199532
min,0.000000,0.000000,11.000000,1.100000e+01,11.000000,0.010000,0.008571,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,17.000000,2.000000e+01,19.000000,60.000000,26.040000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,31.000000,4.100000e+01,39.000000,138.770000,67.414870,1.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,70.000000,1.070000e+02,99.000000,270.000000,110.796645,1.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,31740.000000,1.414710e+06,90294.000000,77755.181818,42813.558857,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
_deepnote_run_altair(df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"Avg_Mdcr_Alowd_Amt","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"Avg_Sbmtd_Chrg","type":"quantitative","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f3594ae2-e1b0-4476-8420-ffda39dfce6e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>